https://www.youtube.com/playlist?list=PLzvRQMJ9HDiTqZmbtFisdXFxul5k0F-Q4

In [1]:
import hashlib
from datetime import datetime

## Part 1: A Simple Blockchain

In [84]:
class Block:
    def __init__(self, timestamp, data, previous_hash = ''):
        self.timestamp = timestamp
        self.data = str(data)
        self.previous_hash = previous_hash
        self.nonce = 0
        self.blockhash = self.calculate_hash()
        
    def calculate_hash(self):
        concat = self.previous_hash + self.timestamp + self.data + str(self.nonce)
        return hashlib.sha224(concat.encode('utf-8')).hexdigest()
    
    def get_block_data(self):
        return {
            'timestamp': self.timestamp,
            'data': self.data,
            'previous_hash': self.previous_hash,
            'hash': self.blockhash
        }
   

class BlockChain:
    def __init__(self):
        self.chain = []
        self.chain.append(self.create_genesis_block())

    def create_genesis_block(self):
        return Block(get_timestamp(), ["Genesis Block"], "0")
    
    def get_latest_block(self):
        return self.chain[-1]
    
    def add_block(self, new_block):
        new_block.previous_hash = self.get_latest_block().blockhash
        new_block.blockhash = new_block.calculate_hash()
        self.chain.append(new_block)
        
    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current = self.chain[i]
            previous = self.chain[i-1]
            if current.blockhash != current.calculate_hash():
                return False
            if current.previous_hash != previous.blockhash:
                return False
        return True
            

def get_timestamp():
    date = datetime.now()
    # convert datetime to timestamp
    timestamp = datetime.timestamp(date)
    return str(timestamp)



In [80]:
bc = BlockChain()
d = {
    'Name': 'Noureddin Sadawi',
    "Age": 43
}
bc.add_block(Block(get_timestamp(), d))
bc.add_block(Block(get_timestamp(), d))

In [81]:
bc.is_chain_valid()

True

In [82]:
bc.chain[1].data = "Koko"
bc.chain[1].blockhash = bc.chain[1].calculate_hash()

In [83]:
bc.chain[1].blockhash

'dcac6995741a716884e5fc9b326a769921105ed573b70c1b40933311'

## Part 2: Proof of Work

You have to prove that you have put in a lot of computing power into making a Block (this is called Mining).

BitCoin: requires the hash of a block to begin with a certain number of 0's

In [32]:
class Block:
    def __init__(self, timestamp, data, previous_hash = ''):
        self.timestamp = timestamp
        self.data = str(data)
        self.previous_hash = previous_hash
        self.nonce = 0
        self.blockhash = self.calculate_hash()
        
    def calculate_hash(self):
        concat = self.previous_hash + self.timestamp + self.data + str(self.nonce)
        return hashlib.sha224(concat.encode('utf-8')).hexdigest()
    
    def mine_block(self, difficulty):
        # here we see the benefit of the nonce, we increment it until we get the right hash
        # this is because the hash can always be the same with the same block contents
        while self.blockhash[:difficulty] != "0" * difficulty:
            self.nonce = self.nonce + 1
            self.blockhash = self.calculate_hash()
    
    def get_block_data(self):
        return {
            'timestamp': self.timestamp,
            'data': self.data,
            'previous_hash': self.previous_hash,
            'hash': self.blockhash
        }
   

class BlockChain:
    def __init__(self, difficulty=2):
        self.chain = []
        self.chain.append(self.create_genesis_block())
        self.difficulty = difficulty

    def create_genesis_block(self):
        return Block(get_timestamp(), ["Genesis Block"], "0")
    
    def get_latest_block(self):
        return self.chain[-1]
    
    def add_block(self, new_block):
        new_block.previous_hash = self.get_latest_block().blockhash
        new_block.mine_block(self.difficulty)
        #new_block.blockhash = new_block.calculate_hash()
        self.chain.append(new_block)
        
    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current = self.chain[i]
            previous = self.chain[i-1]
            if current.blockhash != current.calculate_hash():
                return False
            if current.previous_hash != previous.blockhash:
                return False
        return True
            

def get_timestamp():
    date = datetime.now()
    # convert datetime to timestamp
    timestamp = datetime.timestamp(date)
    return str(timestamp)



In [33]:
bc = BlockChain(difficulty=4)
d = {
    'Name': 'Noureddin Sadawi',
    "Age": 43
}
bc.add_block(Block(get_timestamp(), d))
bc.add_block(Block(get_timestamp(), d))

In [34]:
bc.chain[1].blockhash

'00000260150082384d8b5805701ef518f5c604548675610b8d5bc602'

## Part 3: Mining Rewards and Transactions
1- Now a block can contain more than one transaction

2- Add rewards for mining


When you start a crypto-currency, you need to have virtual money or coins, you need to introduce them somewhere in the system so that users can get coins. Mining rewards steadily add new coins into the system.

In [85]:
class Transaction:
    def __init__(self, from_address, to_address, amount):
        self.from_address = from_address
        self.to_address = to_address
        self.amount = amount
    
class Block:
    def __init__(self, timestamp, transactions, previous_hash = ''):
        self.timestamp = timestamp
        self.transactions = transactions
        self.previous_hash = previous_hash
        self.nonce = 0
        self.blockhash = self.calculate_hash()
        
    def calculate_hash(self):
        concat = self.previous_hash + self.timestamp + str(self.transactions) + str(self.nonce)
        return hashlib.sha224(concat.encode('utf-8')).hexdigest()
    
    def mine_block(self, difficulty):
        # here we see the benefit of the nonce, we increment it until we get the right hash
        # this is because the hash can always be the same with the same block contents
        while self.blockhash[:difficulty] != "0" * difficulty:
            self.nonce = self.nonce + 1
            self.blockhash = self.calculate_hash()
    
    def get_block_data(self):
        return {
            'timestamp': self.timestamp,
            'transactions': self.transactions,
            'previous_hash': self.previous_hash,
            'hash': self.blockhash
        }
   

class BlockChain:
    def __init__(self, difficulty=2):
        self.chain = []
        self.chain.append(self.create_genesis_block())
        self.difficulty = difficulty
        self.pending_transactions = []
        self.mining_reward = 100

    def create_genesis_block(self):
        return Block(get_timestamp(), [Transaction('','',0)], "0")

    
    def get_latest_block(self):
        return self.chain[-1]
    
    #def add_block(self, new_block):
    #    new_block.previous_hash = self.get_latest_block().blockhash
    #    new_block.mine_block(self.difficulty)
    #    #new_block.blockhash = new_block.calculate_hash()
    #    self.chain.append(new_block)
    def mine_pending_transactions(self, mining_reward_address):
        # in a real crypto currency it's not practical to use all pending_transactions
        # there is soooo many of them
        # you might choose which ones to include
        new_block = Block(get_timestamp(), self.pending_transactions)
        new_block.mine_block(self.difficulty)
        print('Block mined!')
        self.chain.append(new_block)
        # reset pending_transactions
        self.pending_transactions = [
            Transaction('', mining_reward_address, self.mining_reward)
        ]
    
        
    def create_transaction(self, transaction):
        self.pending_transactions.append(transaction)
        
    def get_balance(self, address):
        balance = 0
        for b in self.chain:
            for trans in b.transactions:
                if trans.from_address == address:
                    balance = balance - trans.amount
                if trans.to_address == address:
                    balance = balance + trans.amount
        return balance
        
    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current = self.chain[i]
            previous = self.chain[i-1]
            if current.blockhash != current.calculate_hash():
                return False
            if current.previous_hash != previous.blockhash:
                return False
        return True
            

def get_timestamp():
    date = datetime.now()
    # convert datetime to timestamp
    timestamp = datetime.timestamp(date)
    return str(timestamp)



In [86]:
bc = BlockChain()


<class 'list'>


In [87]:
bc.pending_transactions

[]

In [88]:
## in reality address 1 and address 2 would be the public key of someone's wallet
bc.create_transaction(Transaction('address1','address2',100))
bc.create_transaction(Transaction('address2','address1',50))

In [89]:
bc.mine_pending_transactions('noureddin')

<class 'list'>
Block mined!


In [90]:
bc.get_balance('noureddin')

<class '__main__.Block'>
<class '__main__.Transaction'>
<class '__main__.Block'>
<class '__main__.Transaction'>
<class '__main__.Transaction'>


0

In [91]:
bc.mine_pending_transactions('noureddin')

<class 'list'>
Block mined!


In [92]:
bc.get_balance('noureddin')

<class '__main__.Block'>
<class '__main__.Transaction'>
<class '__main__.Block'>
<class '__main__.Transaction'>
<class '__main__.Transaction'>
<class '__main__.Block'>
<class '__main__.Transaction'>


100

In [1]:
!pip install pycoin

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for pycoin: filename=pycoin-0.91.20210515-py3-none-any.whl size=188924 sha256=c5da4e5b35569fee8c5e4a2918e81dd1628796d66855f8b7e59e74853131a213
  Stored in directory: c:\users\gaming\appdata\local\pip\cache\wheels\eb\0c\55\facd488ef94f6ac82da94b2b4a6648bc6ff3ca7fef634a535f
Successfully built pycoin


## 4. Signing Transactions

Needed for security
* The problem with the current version is that anyone can make any transaction that they want.
* Effectively you can spend coins that aren't yours.
* To solve that we'll make it mandatory for transactions to be signed with a private and public key.
* That way you can only spend coins in a wallet if you have its private key

In [88]:
class Transaction:
    ## Now we need to sign the transaction
    # and we need a method to check if this signature is valid
    ## we're going to sign the hash of the transaction
    def __init__(self, from_address, to_address, amount):
        self.from_address = from_address
        self.to_address = to_address
        self.amount = amount
        self.signature = None
    
    def calculate_hash(self):
        ## we're going to sign the hash of the transaction
        ## Creates a SHA256 hash of the transaction
        concat = self.from_address + self.to_address + str(self.amount)
        return hashlib.sha224(concat.encode('utf-8')).hexdigest()
    
    
    ## Signs a transaction with the given signingKey (which is an Elliptic keypair
    ## object that contains a private key). The signature is then stored inside the
    ## transaction object and later stored on the blockchain.
    def sign_transaction(self, signing_key):
        ## check if your public key == the from address
        ## you can only spend coins from the wallet you have the private key of!
        ## because the priv key is linked to the public key, that means the from add
        ## in the transaction has to equal your public key
        if signing_key.get_verifying_key().to_string().hex() != self.from_address:
            raise ValueError('Error. You cannot sign transactions for other wallets!')
        
        ## Calculate the hash of this transaction, sign it with the key
        ## and store it inside the transaction obect
        trans_hash = self.calculate_hash()
        signature = signing_key.sign(bytes(trans_hash.encode('utf-8')))
        self.signature = signature
        
    ## Checks if the signature is valid (transaction has not been tampered with).
    ## It uses the from_address as the public key.
    def is_valid(self):
        ## check if transaction has been correctly signed
        if self.from_address == None: ## mining reward transaction (from_address is not filled in)
            ## mining reward transactions are not signed but they are valid
            return True
        ## if transaction needs a signature, make sure there is a signature
        ## (from_address is filled in)
        ## You can only send a transaction from the wallet that is linked to your
        ## key. So here we check if the from_address matches your publicKey
        if (self.signature == None) or (len(self.signature) == 0):
            raise ValueError('Error. No signature in this transaction!')
        
        ## check if transaction was signed with the correct key
        ## remember the from_address is a public key
        ## from https://www.py4u.net/discuss/164146
        ## create a verification key from from_address (remember it's a public key)
        
        vk = ecdsa.VerifyingKey.from_string(bytes.fromhex(self.from_address), curve=ecdsa.SECP256k1)
        ## we want to verify that the hash of this block has been signed by this signature
        ##print(self.calculate_hash())
        return vk.verify(self.signature, bytes(self.calculate_hash().encode('utf-8'))) # True or False

class Block:
    def __init__(self, timestamp, transactions, previous_hash = ''):
        self.timestamp = timestamp
        self.transactions = transactions
        self.previous_hash = previous_hash
        self.nonce = 0
        self.blockhash = self.calculate_hash()
        
    ## Returns the SHA256 of this block (by processing all the data stored
    ## inside this block)
    def calculate_hash(self):
        concat = self.previous_hash + self.timestamp + str(self.transactions) + str(self.nonce)
        return hashlib.sha224(concat.encode('utf-8')).hexdigest()
    
    ## Starts the mining process on the block. It changes the 'nonce' until the hash
    ## of the block starts with enough zeros (= difficulty)
    def mine_block(self, difficulty):
        # here we see the benefit of the nonce, we increment it until we get the right hash
        # this is because the hash can always be the same with the same block contents
        while self.blockhash[:difficulty] != "0" * difficulty:
            self.nonce = self.nonce + 1
            self.blockhash = self.calculate_hash()
    
    ## Get details of the this block
    def get_block_data(self):
        return {
            'timestamp': self.timestamp,
            'transactions': self.transactions,
            'previous_hash': self.previous_hash,
            'hash': self.blockhash
        }
    
    ## Validates all the transactions inside this block (signature + hash) and
    ## returns true if everything checks out. False if the block is invalid.
    def has_valid_transactions(self):
        ## verify all all transactions in the current block
        ## iteratue thru all transactions in the current block
        for trans in self.transactions:
            if not trans.is_valid():
                return False
        return True
    

class BlockChain:
    def __init__(self, difficulty=2):
        self.chain = []
        self.chain.append(self.create_genesis_block())
        self.difficulty = difficulty
        self.pending_transactions = []
        self.mining_reward = 100

    ## Create the very first block on the chain
    def create_genesis_block(self):
        return Block(get_timestamp(), [Transaction('','',0)], "0")

    ## Returns the latest block on our chain. Useful when you want to create a
    ## new Block and you need the hash of the previous Block.
    def get_latest_block(self):
        return self.chain[-1]
    
    #def add_block(self, new_block):
    #    new_block.previous_hash = self.get_latest_block().blockhash
    #    new_block.mine_block(self.difficulty)
    #    #new_block.blockhash = new_block.calculate_hash()
    #    self.chain.append(new_block)
    
    
    ## Takes all the pending transactions, puts them in a Block and starts the
    ## mining process. It also adds a transaction to send the mining reward to
    ## the given address.
    def mine_pending_transactions(self, mining_reward_address):
        # in a real crypto currency it's not practical to use all pending_transactions
        # there is soooo many of them
        # you might choose which ones to include
        
        # reset pending_transactions
        self.pending_transactions.append(
            Transaction(None, mining_reward_address, self.mining_reward)
        )
        # create a new block by adding the pending_transactions and the hash of the latest block
        new_block = Block(get_timestamp(), self.pending_transactions, self.get_latest_block().blockhash)
        new_block.mine_block(self.difficulty)
        # print('Block mined!')
        self.chain.append(new_block)
        self.pending_transactions = []
        
    
    ## Add a new transaction to the list of pending transactions (to be added
    ## next time the mining process starts). This verifies that the given
    ## transaction is properly signed.
    def add_transaction(self, transaction):
        # we're not creating a transaction, we're receiving it and adding it to the pending_transactions list
        if (transaction.from_address == None) or (transaction.to_address == None):
            raise ValueError('Error. Transaction must have from and to addresses!')
        # make sure transaction is valid (erify it)
        if not transaction.is_valid():
            raise ValueError('Error. Cannot add invalid transaction to the chain!')
        
        ## Making sure that the amount is > 0
        if transaction.amount <= 0:
            raise ValueError('Error. Transaction amount should be greater than 0!')
            
        ## Making sure that the amount sent is not greater than existing balance
        #if self.get_balance(transaction.from_address) < transaction.amount:
        #    print(self.get_balance(transaction.from_address))
        #    print(transaction.amount)
        #    raise ValueError('Error. Not enough balance!')
          
        self.pending_transactions.append(transaction)
        
        
    ## Returns the balance of a given wallet address.    
    def get_balance(self, address):
        balance = 0
        for b in self.chain:
            for trans in b.transactions:
                if trans.from_address == address:
                    balance = balance - trans.amount
                if trans.to_address == address:
                    balance = balance + trans.amount
        return balance
        
    ## Loops over all the blocks in the chain and verify if they are properly
    ## linked together and nobody has tampered with the hashes. By checking
    ## the blocks it also verifies the (signed) transactions inside of them.
    def is_chain_valid(self):
        ## TODO?: Check if the Genesis block hasn't been tampered with by comparing
        ## the output of create_genesis_block with the first block on our chain??
    
        # Goes thru all blocks in the chain and verifies that the hashes are correct
        # and each block links to the previous block
        # we'll also need to verify that all the transactions in the current block are valid
        for i in range(1, len(self.chain)):
            #Check the remaining blocks on the chain to see if there hashes and
            # signatures are correct
            current = self.chain[i]
            previous = self.chain[i-1]
            
            if not current.has_valid_transactions():
                return False
            
            if current.blockhash != current.calculate_hash():
                return False
            if current.previous_hash != previous.calculate_hash():
                return False
        return True
            
## Returns a timestamp of NOW
def get_timestamp():
    date = datetime.now()
    # convert datetime to timestamp
    timestamp = datetime.timestamp(date)
    return str(timestamp)



In [79]:
#!pip install ecdsa

In [89]:
import ecdsa
import base64

sk = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1) #this is your sign (private key)
private_key = sk.to_string().hex() #convert your private key to hex
vk = sk.get_verifying_key() #this is your verification key (public key)
public_key = vk.to_string().hex()#this is the wallet address

In [90]:
coin = BlockChain()

tx1 = Transaction(public_key, 'someone else public key', 10)
tx1.sign_transaction(sk)
coin.add_transaction(tx1)
coin.mine_pending_transactions(public_key)

In [91]:
coin.get_balance(public_key)

90

In [92]:
coin.is_chain_valid()

True

In [93]:
(coin.chain[0].get_block_data())

{'timestamp': '1632420235.080689',
 'transactions': [<__main__.Transaction at 0x153d369c040>],
 'previous_hash': '0',
 'hash': '6d812d29361dfeb979d104768bcef1c4579c828ab9524429fdbe0386'}

In [94]:
public_key

'b59ac0b555218094314409e39f51ba56ee5ea475d2989c0cbfc33c1105a0c61f72bb2df7fb0f901137cf61303e81615046adb1be24b333bb768ff08d28e74d1b'

In [96]:
print(coin.chain[0].transactions[0].to_address)